<a href="https://colab.research.google.com/github/FevDer/Deep_Learning_Project/blob/main/Cloth_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Cloth Classifier using CNN in pytorch

In [ ]:
!pip install torchmetrics


In [3]:
import torch
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchmetrics import Accuracy, Precision, Recall
import matplotlib.pyplot as plt
from torchvision import datasets
import torchvision.transforms as transforms

In [4]:
train_data = datasets.FashionMNIST(root='./data', train=True, download=True, transform = transforms.ToTensor())
test_data = datasets.FashionMNIST(root='./data', train=False, download=True, transform=transforms.ToTensor())

100%|██████████| 26421880/26421880 [00:02<00:00, 11718187.69it/s]


Extracting ./data/FashionMNIST/raw/train-images-idx3-ubyte.gz to ./data/FashionMNIST/raw



100%|██████████| 29515/29515 [00:00<00:00, 199279.61it/s]


Extracting ./data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to ./data/FashionMNIST/raw



100%|██████████| 4422102/4422102 [00:01<00:00, 3719743.18it/s]


Extracting ./data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to ./data/FashionMNIST/raw



100%|██████████| 5148/5148 [00:00<00:00, 13665998.10it/s]

Extracting ./data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/FashionMNIST/raw



In [5]:
classes = train_data.classes
num_classes = len(train_data.classes)
print(f"classes: {classes}\nnum_classes: {num_classes}")

classes: ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']
num_classes: 10


In [6]:
num_input_channels = 1
num_output_channels = 16
image_size = train_data[0][0].shape[1]
image_size

28

In [8]:
class MultiClassImageClassifier(nn.Module):

    def __init__(self, num_classes):
        super(MultiClassImageClassifier, self).__init__()
        self.conv1 = nn.Conv2d(num_input_channels, num_output_channels, kernel_size=3, stride=1, padding=1)
        self.relu = nn.ReLU()
        self.maxpool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.flatten = nn.Flatten()

        # Create a fully connected layer
        self.fc = nn.Linear(num_output_channels * (image_size//2)**2, num_classes)

    def forward(self, x):
        # Pass inputs through each layer
        x = self.conv1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        x = self.flatten(x)
        x = self.fc(x)
        return x

In [9]:
dataloader_train = DataLoader(train_data, shuffle=True, batch_size = 10,)

In [10]:
def train_model(optimizer, net, num_epochs):
    num_processed = 0
    criterion = nn.CrossEntropyLoss()
    for epoch in range(num_epochs):
        running_loss = 0.0
        num_processed = 0
        for feature,label in dataloader_train:
            optimizer.zero_grad()
            outputs = net(feature)
            loss = criterion(outputs, label)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            num_processed += len(label)
        print(f"epoch:{epoch}, loss:{running_loss/num_processed}")

    train_loss = running_loss / len(dataloader_train)

In [11]:
net = MultiClassImageClassifier(num_classes)
optimizer = optim.Adam(net.parameters(), lr = 0.001)

In [12]:
train_model(
    optimizer=optimizer,
    net=net,
    num_epochs=1,
)

epoch:0, loss:0.038839613645267675


In [13]:
dataloader_test = DataLoader(test_data,
                            shuffle = False,
                            batch_size =10,)

In [14]:
# Define the metrics
accuracy_metric = Accuracy(task='multiclass', num_classes=num_classes)
precision_metric = Precision(task='multiclass', num_classes=num_classes, average=None)
recall_metric = Recall(task='multiclass', num_classes=num_classes, average=None)

# Run model on test set
net.eval()
predicted = []
for i, (features, labels) in enumerate(dataloader_test):
    output = net.forward(features.reshape(-1, 1, image_size, image_size))
    cat = torch.argmax(output, dim=-1)
    predicted.extend(cat.tolist())
    accuracy_metric(cat, labels)
    precision_metric(cat, labels)
    recall_metric(cat, labels)

# Compute the metrics
accuracy = accuracy_metric.compute().item()
precision = precision_metric.compute().tolist()
recall = recall_metric.compute().tolist()
print('Accuracy:', accuracy)
print('Precision (per class):', precision)
print('Recall (per class):', recall)

Accuracy: 0.8884000182151794
Precision (per class): [0.8105560541152954, 0.9711155295372009, 0.8504183888435364, 0.8838383555412292, 0.777280867099762, 0.9807497262954712, 0.7396449446678162, 0.9431929588317871, 0.955078125, 0.9637826681137085]
Recall (per class): [0.8600000143051147, 0.9750000238418579, 0.8130000233650208, 0.875, 0.8690000176429749, 0.9679999947547913, 0.625, 0.9629999995231628, 0.9779999852180481, 0.9580000042915344]
